#### Imports

In [ ]:
from typing import Union
from typing_extensions import NotRequired, TypedDict
from functools import reduce

from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
import numpy as np
import xarray as xr


from sdm_eurec4a.visulization import (
    adjust_lightness_array,
    set_custom_rcParams,
    handler_map_alpha,
)
from sdm_eurec4a.reductions import mean_and_stderror_of_mean

from sdm_eurec4a import RepositoryPath
import warnings

warnings.filterwarnings("ignore")

In [ ]:
def set_xticks_time(ax):
    xticks = [0, 500, 1000]
    ax.set_xticks(xticks)


def set_yticks_height(ax):
    yticks = [0, 500, 1000, 1500, 2000]
    ax.set_yticks(yticks)


def set_yticks_height_km(ax):
    yticks = [0, 0.5, 1, 1.5, 2]
    ax.set_yticks(yticks)


def set_logxticks_meter(ax):
    xticks = [1e-6, 1e-3]
    xticklabels = [r"$10^{-6}$", r"$10^{-3}$"]
    ax.set_xticks(xticks, xticklabels)


def set_logxticks_micrometer(ax):
    xticks = [1e-3, 1e0, 1e3]
    xticklabels = [r"$10^{-3}$", r"$10^{0}$", r"$10^{3}$"]
    ax.set_xticks(xticks, xticklabels)


def set_logtyticks_psd(ax):
    yticks = [1e0, 1e6]
    yticklabels = [r"$10^0$", r"$10^6$"]
    ax.set_yticks(yticks, yticklabels)


def set_yticks_lwc(ax):
    ax.set_yticks([0, 0.1, 0.2])

In [ ]:
def label_from_attrs(da: xr.DataArray, return_name=True, return_units=True):
    try:
        name = f"{da.attrs['long_name']}"
    except KeyError:
        name = f"{da.name}"

    if "units" in da.attrs:
        units = f"{da.attrs['units']}"
        if "$" not in units:
            units = f"${units}$"

        units = units.replace("$", " ")
        units = rf"$\left[ {units} \right]$"
    else:
        units = "[???]"

    if return_name == True and return_units == True:
        return f"{name} {units}"
    elif return_name == True and return_units == False:
        return f"{name}"
    elif return_name == False and return_units == True:
        return f"{units}"
    else:
        return ""

In [ ]:
plt.style.use("default")
default_colors = set_custom_rcParams()
plt.rcParams.update(
    {
        "axes.spines.top": False,
        "axes.spines.right": False,
        "axes.spines.left": False,
        "axes.spines.bottom": False,
    }
)

dark_colors = adjust_lightness_array(default_colors, amount=0.5)

repo_path = RepositoryPath("levante")()
print(repo_path)

subdata_dir = "output_v3.5"
data_path = Path("/home/m/m301096/CLEO/data/") / subdata_dir

ds_subpath = "combined/eulerian_dataset_combined_v2.nc"

# THE PATH TO THE SCRIPT DIRECTORY
script_dir = Path("/home/m/m301096/repositories/sdm-eurec4a/notebooks/thesis/results/")
print(script_dir)


fig_dir = repo_path / "results" / script_dir.relative_to(repo_path) / subdata_dir / "microphysics"
print(fig_dir)

fig_dir.mkdir(parents=True, exist_ok=True)


(fig_dir / "profiles").mkdir(parents=False, exist_ok=True)
(fig_dir / "comparison").mkdir(parents=False, exist_ok=True)
(fig_dir / "temporal_evolution").mkdir(parents=False, exist_ok=True)


microphysics = (
    "null_microphysics",
    "condensation",
    "collision_condensation",
    "coalbure_condensation_large",
)

/home/m/m301096/repositories/sdm-eurec4a
/home/m/m301096/repositories/sdm-eurec4a/notebooks/thesis/results
/home/m/m301096/repositories/sdm-eurec4a/results/notebooks/thesis/results/output_v3.5/microphysics


# Set time slice to use for temporal mean and median

In [ ]:
time_slice = slice(1500, 3590)  # seconds
radius_split = 45  # µm
radius_slice = slice(1e0, None)  # µm

# Load datasets

In [ ]:
class MicrophysicDict(TypedDict):
    dataset: xr.Dataset
    microphysics: str
    path: Path
    # linestyle: Union[str, tuple]
    color: str


class OptionalDictOfMicrophysicDict(TypedDict):
    null_microphysics: NotRequired[MicrophysicDict]
    condensation: NotRequired[MicrophysicDict]
    collision_condensation: NotRequired[MicrophysicDict]
    coalbure_condensation_large: NotRequired[MicrophysicDict]
    coalbure_condensation_small: NotRequired[MicrophysicDict]


class DictOfMicrophysicDict(TypedDict):
    null_microphysics: MicrophysicDict
    condensation: MicrophysicDict
    collision_condensation: MicrophysicDict
    coalbure_condensation_cke: MicrophysicDict
    coalbure_condensation_large: MicrophysicDict
    coalbure_condensation_small: MicrophysicDict


data_dict = DictOfMicrophysicDict(
    null_microphysics=MicrophysicDict(
        microphysics="Null microphysics",
        path=Path(),
        dataset=xr.Dataset(),
        color="k",
    ),
    condensation=MicrophysicDict(
        microphysics="Condensation/Evaporation",
        path=Path(),
        dataset=xr.Dataset(),
        color="k",
    ),
    collision_condensation=MicrophysicDict(
        microphysics="Coll-coal, cond./evap.",
        path=Path(),
        dataset=xr.Dataset(),
        color="k",
    ),
    coalbure_condensation_cke=MicrophysicDict(
        microphysics="Coll-coal-breakup n by CKE\nand cond./evap.",
        path=Path(),
        dataset=xr.Dataset(),
        color="k",
    ),
    coalbure_condensation_large=MicrophysicDict(
        microphysics="Coll-coal-breakup (n=125)\nand cond./evap.",
        path=Path(),
        dataset=xr.Dataset(),
        color="k",
    ),
    coalbure_condensation_small=MicrophysicDict(
        microphysics="Coll-coal-breakup (n=5)\nand cond./evap.",
        path=Path(),
        dataset=xr.Dataset(),
        color="k",
    ),
)

colors_dict = dict(
    null_microphysics="grey",
    condensation=default_colors[0],
    collision_condensation=default_colors[1],
    coalbure_condensation_cke=default_colors[2],
    coalbure_condensation_large=default_colors[3],
    coalbure_condensation_small=default_colors[4],
)

for mp in data_dict:
    data_dict[mp]["path"] = data_path / f"{mp}" / ds_subpath
    data_dict[mp]["color"] = colors_dict[mp]

for key in data_dict:
    ds = xr.open_dataset(data_dict[key]["path"], chunks={"cloud_id": 2})
    # ds = ds.sel(cloud_id = [18, 301])
    ds.attrs.update(microphysics=data_dict[key]["microphysics"])
    ds.attrs.update(microphysics_short=key)

    data_dict[key]["dataset"] = ds

# ---------------------------------------------------- #
# Reindex the datasets to have the same radius bins
# ---------------------------------------------------- #

combined_radius_bins = reduce(
    np.union1d, [data_dict[mp]["dataset"]["radius_bins"].values for mp in data_dict]
)
fill_value = np.nan
print("Number of radius bins:", len(combined_radius_bins))
print("Fill value:", fill_value)
for mp in data_dict:
    data_dict[mp]["dataset"] = data_dict[mp]["dataset"].reindex(
        radius_bins=combined_radius_bins, fill_value=fill_value
    )

Number of radius bins: 76
Fill value: nan


### All clouds which are simulated

In [ ]:
intersect_cloud_ids = reduce(
    np.intersect1d, [data_dict[key]["dataset"]["cloud_id"].data for key in data_dict]
)
for mp in data_dict:
    print(mp, len(data_dict[mp]["dataset"]["cloud_id"]))

print("\nIntersect:", len(intersect_cloud_ids))
print("cloud_ids:", intersect_cloud_ids)

null_microphysics 94
condensation 78
collision_condensation 83
coalbure_condensation_cke 91
coalbure_condensation_large 93
coalbure_condensation_small 94

Intersect: 66
cloud_ids: [  9  11  18  20  21  22  65  67  68  71  72  73  74  88  94 110 113 114
 130 135 136 142 194 197 198 199 201 203 205 207 208 211 212 213 214 215
 217 218 219 220 221 222 223 224 230 233 235 236 237 292 293 295 296 301
 303 305 306 307 308 309 311 312 314 359 361 362]


In [ ]:
clouds_dict = {
    "222": dict(
        cloud_id=222,
        color="r",
    ),
    "142": dict(
        cloud_id=142,
        color="b",
    ),
}
for key in data_dict:
    ds = data_dict[key]["dataset"]
    for cloud_id in clouds_dict:
        cloud_id = clouds_dict[cloud_id]["cloud_id"]
        is_in = cloud_id in ds["cloud_id"]
        print(f"{cloud_id}, {key}, {is_in}")

222, null_microphysics, True
142, null_microphysics, True
222, condensation, True
142, condensation, True
222, collision_condensation, True
142, collision_condensation, True
222, coalbure_condensation_cke, True
142, coalbure_condensation_cke, True
222, coalbure_condensation_large, True
142, coalbure_condensation_large, True
222, coalbure_condensation_small, True
142, coalbure_condensation_small, True


### Add further variables e.g. latent cooling

In [ ]:
# NOTE: For now, it needs to be divided by 2 s to get values per second Bue to a bug.
def add_variables(
    ds: xr.Dataset, latent_heat_of_condensation: float = 2.265e6, time_slice=time_slice  # J kg-1
):
    # fix attributes
    ds["mass_represented"].attrs["long_name"] = "Mass"

    ds["radius_bins"].attrs["long_name"] = "Radius"
    ds["radius_bins"].attrs["units"] = "$\\mu m$"

    ds["relative_humidity"].attrs["long_name"] = "Relative humidity"
    ds["relative_humidity"].attrs["units"] = "$\\%$"

    ds["gridbox_thickness"] = ds["gridbox_top"] - ds["gridbox_bottom"]
    ds["cloud_altitude"] = ds["gridbox_coord3"].sel(gridbox=ds["max_gridbox"])

    # It seems that xi was stored as an integer. This is not wanted, because nan values will just be large integers.
    ds["xi"] = ds["xi"].astype(float)
    ds["xi"] = ds["xi"].where(ds["xi"] < 1e12)

    ds["xi"].attrs["units"] = "$\\#$"
    ds["xi"].attrs["long_name"] = "Real droplet num. conc."

    ds["xi_per_volume"] = ds["xi"] / ds["gridbox_volume"]
    ds["xi_per_volume"].attrs["long_name"] = ds["xi"].attrs["long_name"]
    ds["xi_per_volume"].attrs["units"] = "$\\# m^{-3}$"

    ds["number_superdroplets_per_volume"] = 1000 * ds["number_superdroplets"] / ds["gridbox_volume"]
    ds["number_superdroplets_per_volume"].attrs["units"] = "$10^3m^{-3}$"
    ds["number_superdroplets_per_volume"].attrs["long_name"] = "Number of superdroplets per volume"

    ds["mass_represented_per_volume"] = 1e3 * ds["mass_represented"] / ds["gridbox_volume"]
    ds["mass_represented_per_volume"].attrs["units"] = "$g m^{-3}$"
    ds["mass_represented_per_volume"].attrs["long_name"] = "Mass"

    ds["evaporation_full"] = 1e3 * ds["massdelta_condensation"].sel(time=time_slice).where(
        ds["sub_cloud_layer_mask"]
    )
    ds["evaporation_full"].attrs["units"] = "$g m^{-3} s^{-1}$"
    ds["evaporation_full"].attrs["long_name"] = "Evaporation rate"
    ds["evaporation_full"].attrs["description"] = "Evaporation rate for all timesteps"

    # ds["evaporation"] = (
    #     ds["evaporation_full"].sel(time=time_slice).mean("time", keep_attrs=True, skipna=True)
    # )
    # ds["evaporation"].attrs["units"] = "$g m^{-3} s^{-1}$"
    # ds["evaporation"].attrs["long_name"] = "Evaporation rate"

    # ds["evaporation"].attrs["units"] = "$g m^{-3} s^{-1}$"
    # ds["evaporation"].attrs["long_name"] = "Evaporation rate"

    ds["latent_heating"] = (
        1e-3
        * ds["evaporation_full"].sel(time=time_slice).mean("time", keep_attrs=True, skipna=True)
        * latent_heat_of_condensation
    )  # kg m-3 s-1 * J kg-1 = W m-3
    ds["latent_heating"].attrs["units"] = "$W m^{-3}$"
    ds["latent_heating"].attrs["long_name"] = "Latent heating"

    ds["latent_heating_full"] = (
        1e-3 * ds["evaporation_full"].sel(time=time_slice) * latent_heat_of_condensation
    )  # kg m-3 s-1 * J kg-1 = W m-3
    ds["latent_heating_full"].attrs["units"] = "$W m^{-3}$"
    ds["latent_heating_full"].attrs["long_name"] = "Latent heating"
    ds["latent_heating_full"].attrs["description"] = "Latent heating for all timesteps"

    ds["latent_heating_mean"] = ds["latent_heating"].mean("gridbox", keep_attrs=True)
    ds["latent_heating_sum"] = (ds["latent_heating"] * ds["gridbox_thickness"]).sum(
        "gridbox", keep_attrs=True
    )
    ds["latent_heating_sum"].attrs["units"] = "$W m^{-2}$"
    ds["latent_heating_sum"].attrs["long_name"] = "Column int. latent heating"

    ds["latent_heating_radius_bins"] = (
        ds["mass_difference_per_volume"].sel(time=time_slice).where(ds["sub_cloud_layer_mask"])
        * latent_heat_of_condensation
    )
    ds["latent_heating_radius_bins"].attrs["units"] = "$W m^{-3}$"
    ds["latent_heating_radius_bins"].attrs["long_name"] = "Latent heating"

    # In order to mask out outliers in the latent heating field,
    # the total latent heating at a timestep for a gridbox should not exceed
    # the minimum or maximum value of the latent heating field from CLEOs monitor output.
    ds["mask_latent_heating"] = (
        ds["latent_heating_radius_bins"] > ds["latent_heating_radius_bins"].quantile(0.05, "time")
    ) & (ds["latent_heating_radius_bins"] < ds["latent_heating_radius_bins"].quantile(0.95, "time"))
    ds["mask_latent_heating"].attrs[
        "description"
    ] = "In order to mask out outliers in the latent heating field, the total latent heating at a timestep for a gridbox should not exceed the minimum or maximum value of the latent heating field from CLEOs monitor output."

    ds["latent_heating_radius_bins_masked"] = ds["latent_heating_radius_bins"].where(
        ds["mask_latent_heating"]
    )

    ds["latent_heating_radius_bins_time_median"] = (
        ds["latent_heating_radius_bins"]
        .sel(time=time_slice)
        .median("time", keep_attrs=True, skipna=True)
    )
    ds["latent_heating_radius_bins_time_median"].attrs["units"] = "$W m^{-3}$"
    ds["latent_heating_radius_bins_time_median"].attrs["long_name"] = "Latent heating"
    ds["latent_heating_radius_bins_time_median"].attrs[
        "description"
    ] = "Median latent heating for all timesteps"


for mp in data_dict:
    ds = data_dict[mp]["dataset"]
    add_variables(ds)

# Analysis of the domain and metrics

## Identify outliers


It seems the outlier is cloud ``296``

In [ ]:
null_microphysics: xr.Dataset = data_dict["null_microphysics"]["dataset"]
condensation: xr.Dataset = data_dict["condensation"]["dataset"]
collision_condensation: xr.Dataset = data_dict["collision_condensation"]["dataset"]
coalbure_condensation_cke: xr.Dataset = data_dict["coalbure_condensation_cke"]["dataset"]
coalbure_condensation_large: xr.Dataset = data_dict["coalbure_condensation_large"]["dataset"]
coalbure_condensation_small: xr.Dataset = data_dict["coalbure_condensation_small"]["dataset"]

# fig, axs = plt.subplots(ncols=5, figsize=(15, 5))
# null_microphysics["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[0])
# condensation["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[1])
# collision_condensation["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[2])
# coalbure_condensation_large["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[3])
# coalbure_condensation_small["mass_represented"].sel(gridbox = 0).sum(dim="radius_bins").plot(ax = axs[4])

# remove outlier cloud_id 296
cloud_id_selection = intersect_cloud_ids[intersect_cloud_ids != 296]

null_microphysics = null_microphysics.sel(cloud_id=cloud_id_selection)
condensation = condensation.sel(cloud_id=cloud_id_selection)
collision_condensation = collision_condensation.sel(cloud_id=cloud_id_selection)
coalbure_condensation_cke = coalbure_condensation_cke.sel(cloud_id=cloud_id_selection)
coalbure_condensation_large = coalbure_condensation_large.sel(cloud_id=cloud_id_selection)
coalbure_condensation_small = coalbure_condensation_small.sel(cloud_id=cloud_id_selection)

# Vertical profiles analysis

### Data of all microphysics

In [ ]:
var = "latent_heating"

datas = [
    condensation,
    collision_condensation,
    # coalbure_condensation_cke,
    coalbure_condensation_small,
    coalbure_condensation_large,
]
# data_colors = default_colors[: len(datas)]
# mean_colors = adjust_lightness_array(data_colors, 0.9)
width_ratios = [2] + [1] * len(datas)

fig, axs = plt.subplots(
    ncols=len(datas) + 1, figsize=(20, 5), sharey=True, sharex=False, width_ratios=width_ratios
)

for idx, data in enumerate(datas):
    mp = data.attrs["microphysics_short"]
    color = colors_dict[mp]
    m_color = adjust_lightness_array([color], 0.9)[0]

    select_ids = data["max_gridbox"] == 33
    data = data.sel(cloud_id=select_ids)
    data = data.where(data["sub_cloud_layer_mask"])
    evaporation = data["latent_heating_full"] * 1e3
    evaporation.attrs["units"] = "m" + data["latent_heating_full"].attrs["units"]
    evaporation.attrs["long_name"] = data["latent_heating_full"].attrs["long_name"]

    evp_cloud_mean, evp_cloud_std = mean_and_stderror_of_mean(
        data=evaporation,
        dims=("time",),
    )
    evp_cloud_mean = evp_cloud_mean.compute()
    evp_cloud_std = evp_cloud_std.compute()

    evp_gridbox_mean, evp_gridbox_std = mean_and_stderror_of_mean(
        data=evp_cloud_mean,
        dims=("cloud_id",),
        data_std=evp_cloud_std,
    )
    evp_gridbox_mean = evp_gridbox_mean.compute()
    evp_gridbox_std = evp_gridbox_std.compute()

    y = data["gridbox_coord3"].median("cloud_id")

    for cloud_id in evp_cloud_mean["cloud_id"]:
        m = evp_cloud_mean.sel(cloud_id=cloud_id)
        s = evp_cloud_std.sel(cloud_id=cloud_id)

        axs[idx + 1].plot(
            m,
            y,
            color=color,
            linestyle="-",
            alpha=0.2,
        )

        axs[idx + 1].fill_betweenx(
            y,
            m + s,
            m - s,
            color=color,
            edgecolor="None",
            alpha=0.1,
        )

    for ax in [axs[idx + 1], axs[0]]:
        ax.fill_betweenx(
            y,
            evp_gridbox_mean - evp_gridbox_std,
            evp_gridbox_mean + evp_gridbox_std,
            color=color,
            alpha=0.1,
            edgecolor="None",
            zorder=1,
        )
        ax.plot(
            evp_gridbox_mean,
            y,
            color=m_color,
            linestyle="-",
            linewidth=2,
            alpha=0.8,
            zorder=10,
            label=data.attrs["microphysics"],
        )

    axs[idx + 1].set_title(data.attrs["microphysics"])

axs[0].axvline(0, color="black", linestyle="--", alpha=0.5)
axs[0].set_title("Different microphysics")
axs[0].set_ylabel("Height [m]")
# axs[0].legend(loc="upper left")

for ax in axs:
    ax.set_xlabel(label_from_attrs(evaporation))
    ax.set_xlim(-300, 0)
    # ax.set_xticks([-75, -50, -25])
# axs[0].set_xlim(-300, -35)
# axs[0].set_xticks([-60, -50, -40])
fig.suptitle("Latent heating for different microphysics")  #
fig.tight_layout()
fig.savefig(fig_dir / "profiles" / "microphysics_latent_heating.svg")
fig.savefig(fig_dir / "profiles" / "microphysics_latent_heating.pdf")

for ax in axs:
    ax.set_xlabel(label_from_attrs(evaporation))
    ax.set_xlim(-100, 0)
    ax.set_xticks([-75, -50, -25])
axs[0].set_xlim(-65, -45)
axs[0].set_xticks([-60, -50, -40])


fig.savefig(fig_dir / "profiles" / "microphysics_latent_heating_zoomed.svg")
fig.savefig(fig_dir / "profiles" / "microphysics_latent_heating_zoomed.pdf")

In [ ]:
datas = [
    condensation,
    collision_condensation,
    # coalbure_condensation_cke,
    coalbure_condensation_small,
    coalbure_condensation_large,
]
width_ratios = [2] + [1] * len(datas)

fig, axs = plt.subplots(
    ncols=len(datas) + 1, figsize=(20, 5), sharey=True, sharex=False, width_ratios=width_ratios
)


for idx, data in enumerate(datas):
    mp = data.attrs["microphysics_short"]
    color = colors_dict[mp]
    m_color = adjust_lightness_array([color], 0.9)[0]

    data = data.where(data["gridbox"] < data["max_gridbox"] - 1)
    evaporation = data["latent_heating_full"] * 1e3
    evaporation.attrs["units"] = "m" + data["latent_heating_full"].attrs["units"]
    evaporation.attrs["long_name"] = data["latent_heating_full"].attrs["long_name"]

    evp_cloud_mean, evp_cloud_std = mean_and_stderror_of_mean(
        data=evaporation,
        dims=("time",),
    )
    evp_cloud_mean = evp_cloud_mean.compute()
    evp_cloud_std = evp_cloud_std.compute()

    evp_gridbox_mean, evp_gridbox_std = mean_and_stderror_of_mean(
        data=evp_cloud_mean,
        dims=("cloud_id",),
        data_std=evp_cloud_std,
    )
    evp_gridbox_mean = evp_gridbox_mean.compute()
    evp_gridbox_std = evp_gridbox_std.compute()

    y = data["gridbox"].astype(int) * 20 - 10

    for cloud_id in evp_cloud_mean["cloud_id"]:
        m = evp_cloud_mean.sel(cloud_id=cloud_id)
        s = evp_cloud_std.sel(cloud_id=cloud_id)

        axs[idx + 1].plot(
            m,
            y,
            color=color,
            linestyle="-",
            alpha=0.2,
        )

        axs[idx + 1].fill_betweenx(
            y,
            m + s,
            m - s,
            color=color,
            edgecolor="None",
            alpha=0.1,
        )

    for ax in [axs[idx + 1], axs[0]]:
        ax.fill_betweenx(
            y,
            evp_gridbox_mean - evp_gridbox_std,
            evp_gridbox_mean + evp_gridbox_std,
            color=color,
            alpha=0.1,
            edgecolor="None",
            zorder=1,
        )
        ax.plot(
            evp_gridbox_mean,
            y,
            color=m_color,
            linestyle="-",
            linewidth=2,
            alpha=0.8,
            zorder=10,
        )

    axs[idx + 1].set_title(data.attrs["microphysics"])

axs[0].axvline(0, color="black", linestyle="--", alpha=0.5)
axs[0].set_title("Different microphysics")
axs[0].set_ylabel("Height [m]")
axs[0].legend(loc="upper left")

for ax in axs:
    ax.set_xlabel(label_from_attrs(evaporation))
    ax.set_xlim(-100, 0)
    ax.set_xticks([-75, -50, -25])
axs[0].set_xlim(-65, -15)
axs[0].set_xticks([-60, -40, -20])
fig.suptitle("Latent heating for different microphysics")  #
fig.tight_layout()
fig.savefig(fig_dir / "profiles" / "all_microphysics_latent_heating.svg")
fig.savefig(fig_dir / "profiles" / "all_microphysics_latent_heating.pdf")

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


### Difference between ``Condensation`` and other microphysics

In [ ]:
var = "latent_heating"


datas = [
    condensation,
    collision_condensation,
    # coalbure_condensation_cke,
    coalbure_condensation_small,
    coalbure_condensation_large,
]
width_ratios = [2] + [1] * len(datas)

fig, axs = plt.subplots(
    ncols=len(datas) + 1, figsize=(20, 5), sharey=True, sharex=False, width_ratios=width_ratios
)

for idx, data in enumerate(datas):
    mp = data.attrs["microphysics_short"]
    color = colors_dict[mp]
    m_color = adjust_lightness_array([color], 0.9)[0]

    select_ids = data["max_gridbox"] == 33
    data = data.sel(cloud_id=select_ids).sel(time=time_slice)
    data = data.where(data["sub_cloud_layer_mask"])
    evaporation = data["latent_heating_full"] * 1e3
    evaporation.attrs["units"] = "m" + data["latent_heating_full"].attrs["units"]
    evaporation.attrs["long_name"] = data["latent_heating_full"].attrs["long_name"]

    compare = condensation["latent_heating_full"] * 1e3

    evp_cloud_mean, evp_cloud_std = mean_and_stderror_of_mean(
        data=evaporation,
        dims=("time",),
    )
    compare_cloud_mean, compare_cloud_std = mean_and_stderror_of_mean(
        data=compare,
        dims=("time",),
    )

    evp_cloud_mean = evp_cloud_mean.compute()
    evp_cloud_std = evp_cloud_std.compute()

    compare_cloud_mean = compare_cloud_mean.compute()
    compare_cloud_std = compare_cloud_std.compute()

    diff_cloud_mean = evp_cloud_mean - compare_cloud_mean
    diff_cloud_std: xr.DataArray = np.sqrt(evp_cloud_std**2 + compare_cloud_std**2)

    diff_gridbox_mean, diff_gridbox_std = mean_and_stderror_of_mean(
        data=diff_cloud_mean,
        dims=("cloud_id",),
        data_std=diff_cloud_std,
    )

    diff_gridbox_mean = diff_gridbox_mean.compute()
    diff_gridbox_std = diff_gridbox_std.compute()

    y = data["gridbox_coord3"].median("cloud_id")

    for cloud_id in diff_cloud_mean["cloud_id"]:
        m = diff_cloud_mean.sel(cloud_id=cloud_id)
        s = diff_cloud_std.sel(cloud_id=cloud_id)

        axs[idx + 1].plot(
            m,
            y,
            color=m_color,
            linestyle="-",
            alpha=0.2,
        )

        axs[idx + 1].fill_betweenx(
            y,
            m - s,
            m + s,
            color=m_color,
            edgecolor="None",
            alpha=0.1,
        )

    for ax in [axs[idx + 1], axs[0]]:
        ax.fill_betweenx(
            y,
            diff_gridbox_mean - diff_gridbox_std,
            diff_gridbox_mean + diff_gridbox_std,
            color=color,
            alpha=0.1,
            edgecolor="None",
            zorder=1,
        )
        ax.plot(
            diff_gridbox_mean,
            y,
            color=m_color,
            linestyle="-",
            linewidth=2,
            alpha=1,
            zorder=10,
        )

    axs[idx + 1].set_title(data.attrs["microphysics"])

axs[0].axvline(0, color="black", linestyle="--", alpha=0.5)
axs[0].set_title("Different microphysics")
axs[0].set_ylabel("Height [m]")
axs[0].legend(loc="upper left")

fig.suptitle("Latent heating profiles compared to condensation only setup")  #

for ax in axs:
    ax.set_xlabel(label_from_attrs(evaporation))
    ax.axvline(0, color="black", linestyle="--", alpha=0.8, zorder=0)

fig.tight_layout()
fig.savefig(fig_dir / "profiles" / "diff_microphysics_latent_heating_nolim.svg")
fig.savefig(fig_dir / "profiles" / "diff_microphysics_latent_heating_nolim.pdf")

for ax in axs:
    ax.set_xlim(-200, 10)

fig.tight_layout()
fig.savefig(fig_dir / "profiles" / "diff_microphysics_latent_heating.svg")
fig.savefig(fig_dir / "profiles" / "diff_microphysics_latent_heating.pdf")

for ax in axs:
    ax.set_xlabel(label_from_attrs(evaporation))
    ax.set_xlim(-5, 10)
    ax.set_xticks(
        [
            -5,
            -0,
            5,
            10,
        ]
    )
axs[0].set_xlim(-0.5, 4.5)
axs[0].set_xticks([0, 2, 4])

fig.tight_layout()
fig.savefig(fig_dir / "profiles" / "diff_microphysics_latent_heating_zoomed.svg")
fig.savefig(fig_dir / "profiles" / "diff_microphysics_latent_heating_zoomed.pdf")

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [ ]:
datas = [
    condensation,
    collision_condensation,
    coalbure_condensation_cke,
    coalbure_condensation_small,
    coalbure_condensation_large,
]
width_ratios = [2] + [1] * len(datas)

fig, axs = plt.subplots(
    ncols=len(datas) + 1, figsize=(20, 5), sharey=True, sharex=False, width_ratios=width_ratios
)

for idx, data in enumerate(datas):
    mp = data.attrs["microphysics_short"]
    color = colors_dict[mp]
    m_color = adjust_lightness_array([color], 0.9)[0]

    data = data.where(data["gridbox"] < data["max_gridbox"] - 1)
    evaporation = data["latent_heating_full"] * 1e3
    evaporation.attrs["units"] = "m" + data["latent_heating_full"].attrs["units"]
    evaporation.attrs["long_name"] = data["latent_heating_full"].attrs["long_name"]

    compare = condensation["latent_heating_full"] * 1e3

    evp_cloud_mean, evp_cloud_std = mean_and_stderror_of_mean(
        data=evaporation,
        dims=("time",),
    )
    compare_cloud_mean, compare_cloud_std = mean_and_stderror_of_mean(
        data=compare,
        dims=("time",),
    )

    evp_cloud_mean = evp_cloud_mean.compute()
    evp_cloud_std = evp_cloud_std.compute()

    compare_cloud_mean = compare_cloud_mean.compute()
    compare_cloud_std = compare_cloud_std.compute()

    diff_cloud_mean = evp_cloud_mean - compare_cloud_mean
    diff_cloud_std: xr.DataArray = np.sqrt(evp_cloud_std**2 + compare_cloud_std**2)

    diff_gridbox_mean, diff_gridbox_std = mean_and_stderror_of_mean(
        data=diff_cloud_mean,
        dims=("cloud_id",),
        data_std=diff_cloud_std,
    )

    diff_gridbox_mean = diff_gridbox_mean.compute()
    diff_gridbox_std = diff_gridbox_std.compute()

    y = data["gridbox"].astype(int) * 20 - 10

    for cloud_id in diff_cloud_mean["cloud_id"]:
        m = diff_cloud_mean.sel(cloud_id=cloud_id)
        s = diff_cloud_std.sel(cloud_id=cloud_id)

        axs[idx + 1].plot(
            m,
            y,
            color=m_color,
            linestyle="-",
            alpha=0.2,
        )

        axs[idx + 1].fill_betweenx(
            y,
            m - s,
            m + s,
            color=m_color,
            edgecolor="None",
            alpha=0.1,
        )

    for ax in [axs[idx + 1], axs[0]]:
        ax.fill_betweenx(
            y,
            diff_gridbox_mean - diff_gridbox_std,
            diff_gridbox_mean + diff_gridbox_std,
            color=color,
            alpha=0.1,
            edgecolor="None",
            zorder=1,
        )
        ax.plot(
            diff_gridbox_mean,
            y,
            color=m_color,
            linestyle="-",
            linewidth=2,
            alpha=1,
            zorder=10,
        )

    axs[idx + 1].set_title(data.attrs["microphysics"])

axs[0].axvline(0, color="black", linestyle="--", alpha=0.5)
axs[0].set_title("Different microphysics")
axs[0].set_ylabel("Height [m]")
axs[0].legend(loc="upper left")
fig.suptitle("Latent heating profiles compared to condensation only setup")  #

for ax in axs:
    ax.set_xlabel(label_from_attrs(evaporation))
    ax.axvline(0, color="black", linestyle="--", alpha=0.8, zorder=0)

fig.tight_layout()
fig.savefig(fig_dir / "profiles" / "all_diff_microphysics_latent_heating_nolim.svg")
fig.savefig(fig_dir / "profiles" / "all_diff_microphysics_latent_heating_nolim.pdf")

for ax in axs:
    ax.set_xlim(-200, 10)

fig.tight_layout()
fig.savefig(fig_dir / "profiles" / "all_diff_microphysics_latent_heating.svg")
fig.savefig(fig_dir / "profiles" / "all_diff_microphysics_latent_heating.pdf")

for ax in axs:
    ax.set_xlabel(label_from_attrs(evaporation))
    ax.set_xlim(-5, 10)
    ax.set_xticks(
        [
            -5,
            -0,
            5,
            10,
        ]
    )
axs[0].set_xlim(-0.5, 4.5)
axs[0].set_xticks([0, 2, 4])

fig.tight_layout()
fig.savefig(fig_dir / "profiles" / "all_diff_microphysics_latent_heating_zoomed.svg")
fig.savefig(fig_dir / "profiles" / "all_diff_microphysics_latent_heating_zoomed.pdf")

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


# Impact of Droplet size

In [ ]:
# cloud_ids_from_gridbox_33 = condensation["cloud_id"].where(
#     condensation["max_gridbox"].compute() == 33, drop=True
# )

# datas = [
#     condensation,
#     collision_condensation,
#     coalbure_condensation_cke,
#     coalbure_condensation_large,
#     coalbure_condensation_small,
# ]

# keys = [data.attrs["microphysics_short"] for data in datas]

# latent_heat_comparison = dict()
# # latent_heat_comparison_masked = dict()
# multiplicity_comparison = dict()
# mass_comparison = dict()
# number_superdroplets_comparison = dict()

# for data in datas:
#     mp = data.attrs["microphysics_short"]
#     da = (
#         1e3
#         * data["latent_heating_radius_bins"]
#         .sel(time=time_slice)
#         .sel(cloud_id=cloud_ids_from_gridbox_33)
#         .mean("time", keep_attrs=True)
#         .compute()
#     )
#     da = da.where(data["sub_cloud_layer_mask"])
#     da.attrs["units"] = "$mW m^{-3}$"
#     da.attrs["long_name"] = "Latent heating"
#     latent_heat_comparison[mp] = da

# # for data in datas:
# #     mp = data.attrs["microphysics_short"]
# #     da = (
# #         1e3
# #         * data["latent_heating_radius_bins_masked"]
# #         .sel(time=time_slice)
# #         .sel(cloud_id=cloud_ids_from_gridbox_33)
# #         .mean("time", keep_attrs=True)
# #         # .compute()
# #     )
# #     da = da.where(data["sub_cloud_layer_mask"])
# #     da.attrs["units"] = "$mW m^{-3}$"
# #     da.attrs["long_name"] = "Latent heating"
# #     latent_heat_comparison_masked[mp] = da

# for data in datas:
#     mp = data.attrs["microphysics_short"]

#     da = (
#         8
#         * data["xi_per_volume"]
#         .sel(time=time_slice)
#         .sel(cloud_id=cloud_ids_from_gridbox_33)
#         .mean("time", keep_attrs=True)
#         .compute()
#     )
#     da = da.where(data["sub_cloud_layer_mask"])
#     da.attrs["units"] = "$ 8000 m^{-3}$"
#     da.attrs["long_name"] = "Multiplicity of SDs"

#     multiplicity_comparison[mp] = da

# for data in datas:
#     mp = data.attrs["microphysics_short"]

#     da = (
#         data["mass_represented_per_volume"]
#         .sel(time=time_slice)
#         .sel(cloud_id=cloud_ids_from_gridbox_33)
#         .mean("time", keep_attrs=True)
#         .compute()
#     )
#     da = da.where(data["sub_cloud_layer_mask"])
#     da.attrs["units"] = "$ 8000 m^{-3}$"
#     da.attrs["long_name"] = "Multiplicity of SDs"

#     multiplicity_comparison[mp] = da


# for data in datas:
#     mp = data.attrs["microphysics_short"]

#     da = (
#         8
#         * data["number_superdroplets_per_volume"]
#         .sel(time=time_slice)
#         .sel(cloud_id=cloud_ids_from_gridbox_33)
#         .mean("time", keep_attrs=True)
#         .compute()
#     )
#     da = da.where(data["sub_cloud_layer_mask"])
#     da.attrs["units"] = "$\\#/8000 m^{-3}$"
#     da.attrs["long_name"] = "Number of SDs"

#     number_superdroplets_comparison[mp] = da

In [ ]:
# datas = [
#     condensation,
#     collision_condensation,
#     coalbure_condensation_cke,
#     coalbure_condensation_large,
#     coalbure_condensation_small,
# ]

# keys = [data.attrs["microphysics_short"] for data in datas]


# fig, axs = plt.subplots(ncols=len(keys) + 1, nrows=4, figsize=(20, 10), sharex=False, sharey=True)

# axs_absolute = axs[0, :]
# axs_anomalie = axs[1, :]


# for idx, mp in enumerate(keys):
#     data = latent_heat_comparison[mp]
#     data = data  # .where(data != 0)
#     diff = data - latent_heat_comparison["condensation"]
#     diff.attrs["units"] = data.attrs["units"]
#     diff.attrs["long_name"] = f"Latent heating difference"

#     pcm1 = axs[0, idx].pcolormesh(
#         data["radius_bins"],
#         data["gridbox"] * 20,
#         data.mean("cloud_id"),
#         shading="nearest",
#         cmap="inferno",
#         vmin=-10,
#         vmax=0,
#     )
#     fig.colorbar(mappable=pcm1, label=label_from_attrs(data))

#     pcm2 = axs[1, idx].pcolormesh(
#         diff["radius_bins"],
#         diff["gridbox"] * 20,
#         diff.mean("cloud_id"),
#         shading="nearest",
#         cmap="RdBu_r",
#         vmin=-1.5,
#         vmax=1.5,
#     )
#     fig.colorbar(mappable=pcm2, label=label_from_attrs(data))

#     nbr = multiplicity_comparison[mp]
#     nbr = nbr.where(nbr <= 1e10)

#     pcm3 = axs[2, idx].pcolormesh(
#         nbr["radius_bins"],
#         nbr["gridbox"] * 20,
#         (nbr - multiplicity_comparison["condensation"]).mean("cloud_id"),
#         shading="nearest",
#         cmap="RdBu_r",
#         norm=mcolors.SymLogNorm(linthresh=1e-1, linscale=1e-6, vmin=-1e3, vmax=1e3),
#     )
#     fig.colorbar(mappable=pcm3, label=label_from_attrs(nbr))

#     nbr = number_superdroplets_comparison[mp]

#     pcm3 = axs[3, idx].pcolormesh(
#         nbr["radius_bins"],
#         nbr["gridbox"] * 20,
#         (nbr - number_superdroplets_comparison["condensation"]).mean("cloud_id"),
#         shading="nearest",
#         cmap="RdBu_r",
#         vmin=-1,
#         vmax=1,
#     )
#     fig.colorbar(mappable=pcm3, label=label_from_attrs(nbr))

#     axs[0, -1].plot(data.sum("radius_bins").mean("cloud_id"), data["gridbox"] * 20, label=mp)

#     axs[1, -1].plot(diff.sum("radius_bins").mean("cloud_id"), diff["gridbox"] * 20, label=mp)

#     axs[0, idx].set_title(data_dict[mp]["microphysics"])


# axs[0, -1].set_title("Sum over all radius bins")


# for ax in axs[:, :-1].flatten():
#     ax.set_xlabel("Radius [µm]")
#     ax.set_xlim(None, None)

# for ax in axs[:, :-1].flatten():
#     ax.set_xscale("log")

# for ax in axs.flatten():
#     ax.set_ylim(0, 650)


# axs[0, 0].set_ylabel("Latent heating\nAbsolute values\n\nHeight [m]")
# axs[1, 0].set_ylabel("Latent heating\nDiff to condensation\n\nHeight [m]")
# axs[2, 0].set_ylabel("Multiplicity\n\n\nHeight [m]")
# axs[3, 0].set_ylabel("Number SDs\n\n\nHeight [m]")

# fig.tight_layout()
# fig.savefig(fig_dir / "latent_heating_comparison_mean.png", dpi=400)

In [ ]:
# keys = [
#     "condensation",
#     "collision_condensation",
#     "coalbure_condensation_large",
#     "coalbure_condensation_small",
# ]

# fig, axs = plt.subplots(ncols=len(keys) + 1, nrows=4, figsize=(20, 10), sharex=False, sharey=True)

# axs_absolute = axs[0, :]
# axs_anomalie = axs[1, :]


# for idx, mp in enumerate(keys):
#     data = latent_heat_comparison[mp]
#     data = data  # .where(data != 0)
#     diff = data - latent_heat_comparison["condensation"]
#     diff.attrs["units"] = data.attrs["units"]
#     diff.attrs["long_name"] = f"Latent heating difference"

#     pcm1 = axs[0, idx].pcolormesh(
#         data["radius_bins"],
#         data["gridbox"] * 20,
#         data.median("cloud_id"),
#         shading="nearest",
#         cmap="inferno",
#         vmin=-10,
#         vmax=0,
#     )
#     fig.colorbar(mappable=pcm1, label=label_from_attrs(data))

#     pcm2 = axs[1, idx].pcolormesh(
#         diff["radius_bins"],
#         diff["gridbox"] * 20,
#         diff.median("cloud_id"),
#         shading="nearest",
#         cmap="RdBu_r",
#         vmin=-1.5,
#         vmax=1.5,
#     )
#     fig.colorbar(mappable=pcm2, label=label_from_attrs(data))

#     nbr = multiplicity_comparison[mp]
#     nbr = nbr.where(nbr <= 1e10)

#     pcm3 = axs[2, idx].pcolormesh(
#         nbr["radius_bins"],
#         nbr["gridbox"] * 20,
#         (nbr - multiplicity_comparison["condensation"]).median("cloud_id"),
#         shading="nearest",
#         cmap="RdBu_r",
#         norm=mcolors.SymLogNorm(linthresh=1e-3, linscale=1e-6, vmin=-1e3, vmax=1e3),
#     )
#     fig.colorbar(mappable=pcm3, label=label_from_attrs(nbr))

#     nbr = number_superdroplets_comparison[mp]

#     pcm3 = axs[3, idx].pcolormesh(
#         nbr["radius_bins"],
#         nbr["gridbox"] * 20,
#         (nbr - number_superdroplets_comparison["condensation"]).median("cloud_id"),
#         shading="nearest",
#         cmap="RdBu_r",
#         vmin=-1,
#         vmax=1,
#     )
#     fig.colorbar(mappable=pcm3, label=label_from_attrs(nbr))

#     axs[0, -1].plot(data.sum("radius_bins").median("cloud_id"), data["gridbox"] * 20, label=mp)

#     axs[1, -1].plot(diff.sum("radius_bins").median("cloud_id"), diff["gridbox"] * 20, label=mp)

#     axs[0, idx].set_title(data_dict[mp]["microphysics"])


# axs[0, -1].set_title("Sum over all radius bins")


# for ax in axs[:, :-1].flatten():
#     ax.set_xlabel("Radius [µm]")
#     ax.set_xlim(None, None)

# for ax in axs[:, :-1].flatten():
#     ax.set_xscale("log")

# for ax in axs.flatten():
#     ax.set_ylim(0, 650)


# axs[0, 0].set_ylabel("Latent heating\nAbsolute values\n\nHeight [m]")
# axs[1, 0].set_ylabel("Latent heating\nDiff to condensation\n\nHeight [m]")
# axs[2, 0].set_ylabel("Multiplicity\n\n\nHeight [m]")
# axs[3, 0].set_ylabel("Number SDs\n\n\nHeight [m]")


# fig.tight_layout()
# fig.savefig(fig_dir / "latent_heating_comparison_median.png", dpi=400)

# Comparison at selected heights

In [ ]:
selected_gridboxes = (
    # condensation["max_gridbox"].compute() - 1,
    condensation["max_gridbox"].compute() - 2,
    15,
    1,
)

labels = (
    # "Cloud base - 20m",
    "Cloud base - 30m",
    f"{condensation['gridbox_coord3'].sel(gridbox = 15).isel(cloud_id= 0).values} m",
    # f"{condensation['gridbox_coord3'].sel(gridbox = 5).isel(cloud_id= 0).values} m",
    f"{condensation['gridbox_coord3'].sel(gridbox = 1).isel(cloud_id= 0).values} m",
)

needed_gridboxes = np.unique(np.hstack(selected_gridboxes))
needed_gridboxes

array([ 1, 15, 23, 24, 25, 29, 31, 32, 33, 34, 35, 45, 49, 50])

In [ ]:
datas = [
    condensation,
    collision_condensation,
    # coalbure_condensation_cke,
    coalbure_condensation_large,
    coalbure_condensation_small,
]

keys = [data.attrs["microphysics_short"] for data in datas]


latent_heat_comparison_all = dict()
for data in datas:
    mp = data.attrs["microphysics_short"]
    print(mp)
    da = (
        1e3
        * data["latent_heating_radius_bins"]
        .sel(time=time_slice)
        .sel(cloud_id=cloud_id_selection)
        .sel(gridbox=needed_gridboxes)
        .median("time", keep_attrs=True)
        .fillna(0)
        .compute()
    )
    da = da.where(data["sub_cloud_layer_mask"])
    da.attrs["units"] = "$mW m^{-3}$"
    da.attrs["long_name"] = "Latent heating"
    latent_heat_comparison_all[mp] = da

multiplicity_comparison_all = dict()
for data in datas:
    mp = data.attrs["microphysics_short"]
    print(mp)
    da = (
        data["xi_per_volume"]
        .sel(time=time_slice)
        .sel(cloud_id=cloud_id_selection)
        .sel(gridbox=needed_gridboxes)
        .mean("time", keep_attrs=True)
        .fillna(0)
        .compute()
    )
    da = da.where(data["sub_cloud_layer_mask"])
    da.attrs["units"] = data["xi_per_volume"].attrs["units"]
    da.attrs["long_name"] = data["xi_per_volume"].attrs["long_name"]

    multiplicity_comparison_all[mp] = da

number_superdroplets_comparison_all = dict()
for data in datas:
    mp = data.attrs["microphysics_short"]
    print(mp)
    da = (
        8
        * data["number_superdroplets_per_volume"]
        .sel(time=time_slice)
        .sel(cloud_id=cloud_id_selection)
        .sel(gridbox=needed_gridboxes)
        .mean("time", keep_attrs=True)
        .fillna(0)
        .compute()
    )
    da = da.where(data["sub_cloud_layer_mask"])
    da.attrs["units"] = "$\\left(8000 m^{3} \\right)^{-1}$"
    da.attrs["long_name"] = "Number of SDs"

    number_superdroplets_comparison_all[mp] = da

mass_comparison_all = dict()
for data in datas:
    mp = data.attrs["microphysics_short"]
    print(mp)
    da = (
        1e3
        * data["mass_represented_per_volume"]
        .sel(time=time_slice)
        .sel(cloud_id=cloud_id_selection)
        .sel(gridbox=needed_gridboxes)
        .mean("time", keep_attrs=True)
        .fillna(0)
        .compute()
    )
    da = da.where(data["sub_cloud_layer_mask"])
    da.attrs["units"] = "m" + data["mass_represented_per_volume"].attrs["units"].replace("$", "")
    da.attrs["long_name"] = data["mass_represented_per_volume"].attrs["long_name"]

    mass_comparison_all[mp] = da

condensation
collision_condensation
coalbure_condensation_large
coalbure_condensation_small
condensation
collision_condensation
coalbure_condensation_large
coalbure_condensation_small
condensation
collision_condensation
coalbure_condensation_large
coalbure_condensation_small
condensation
collision_condensation
coalbure_condensation_large
coalbure_condensation_small


In [ ]:
def plot_mean_median_iqr(ax, data, dim, x_dim):
    x = data[x_dim]

    ax.fill_between(
        x,
        data.quantile(0.75, dim),
        data.quantile(0.25, dim),
        color="red",
        alpha=0.2,
        label="IQR",
        zorder=3,
    )

    ax.plot(x, data.T, linestyle="-", marker="None", color=[0.75, 0.75, 0.75], alpha=0.2, zorder=1)

    ax.plot(
        x,
        data.mean(dim),
        color="darkorange",
        linestyle="None",
        marker="x",
        alpha=1,
        label="Mean",
        zorder=5,
    )
    ax.plot(
        x,
        data.median(dim),
        color="red",
        linestyle="None",
        marker="+",
        alpha=1,
        label="Median",
        zorder=5,
    )

    return ax

#### Latent heating

In [ ]:
datas = [
    condensation,
    collision_condensation,
    # coalbure_condensation_cke,
    coalbure_condensation_small,
    coalbure_condensation_large,
]

keys = [data.attrs["microphysics_short"] for data in datas]


comparison_dict = latent_heat_comparison_all
compare_key = "condensation"

width_ratios = np.ones(len(keys) + 1)
width_ratios[0] = 0.1
width_ratios = list(width_ratios)

fig1, axs1 = plt.subplots(
    ncols=len(keys) + 1,
    nrows=len(selected_gridboxes),
    figsize=(15, 10),
    sharex=False,
    sharey=False,
    width_ratios=width_ratios,
)
fig2, axs2 = plt.subplots(
    ncols=len(keys) - 1 + 1,
    nrows=len(selected_gridboxes),
    figsize=(12, 10),
    sharex=False,
    sharey=False,
    width_ratios=width_ratios[:-1],
)

for idx_mp, mp in enumerate(keys):
    idx_mp += 1
    for idx_height, sel_gridbox in enumerate(selected_gridboxes):
        data = comparison_dict[mp]
        data = data.sel(gridbox=sel_gridbox).compute()
        diff = data - comparison_dict[compare_key].sel(gridbox=sel_gridbox).compute()
        diff.attrs["units"] = data.attrs["units"]
        diff.attrs["long_name"] = data.attrs["long_name"]

        plot_mean_median_iqr(ax=axs1[idx_height, idx_mp], data=data, dim="cloud_id", x_dim="radius_bins")
        axs1[0, idx_mp].set_title(data_dict[mp]["microphysics"])

        if idx_mp > 1:
            plot_mean_median_iqr(
                ax=axs2[idx_height, idx_mp - 1], data=diff, dim="cloud_id", x_dim="radius_bins"
            )
            axs2[0, idx_mp - 1].set_title(data_dict[mp]["microphysics"])


axs1[0, 1].legend(loc="upper left")
axs2[0, 1].legend(loc="upper left")

for idx, label in enumerate(labels):
    axs1[idx, 0].set_ylabel(label, fontsize=14)
    axs2[idx, 0].set_ylabel(label, fontsize=14)

    axs1[idx, 0].set_xticks([])
    axs2[idx, 0].set_xticks([])
    axs1[idx, 0].set_yticks([])
    axs2[idx, 0].set_yticks([])

# data figure
for ax in axs1[:, 1:].flatten():
    ax.set_xlabel("Radius [µm]")
    ax.set_xscale("log")
    ax.set_yscale("linear")
    ax.set_ylim(-15, 5)


# anomaly figure
for ax in axs2[:, 1:].flatten():
    ax.set_xlabel("Radius [µm]")
    ax.set_xscale("log")
    ax.set_yscale("linear")
    ax.set_ylim(-2, 2)

for ax in axs1[:, 1].flatten():
    ax.set_ylabel(label_from_attrs(data))
for ax in axs2[:, 1].flatten():
    ax.set_ylabel(label_from_attrs(diff))

for ax in axs2[:, 1:].flatten():
    ax.axhline(0, color="grey", linestyle="--", alpha=0.5)

fig1.suptitle(f"{label_from_attrs(data)}")
fig2.suptitle(f"Anomalies of {label_from_attrs(diff)} compared to condensation only run")

for fig in [fig1, fig2]:
    fig.tight_layout()

for ax in axs1[:, 1:].flatten():
    ax.set_xlim(10, None)
for ax in axs2[:, 1:].flatten():
    ax.set_xlim(10, None)

# fig1.savefig(fig_dir / "comparison" / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes.png", dpi=400)
fig1.savefig(
    fig_dir
    / "comparison"
    / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes.pdf"
)
# fig2.savefig(fig_dir / "comparison" / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_anomalies.png", dpi=400)
fig2.savefig(
    fig_dir
    / "comparison"
    / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_anomalies.pdf"
)


for ax in axs1[:, 1:].flatten():
    ax.set_xlim(10, None)
for ax in axs2[:, 1:].flatten():
    ax.set_xlim(10, None)

# fig1.savefig(fig_dir / "comparison" / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_zoom.png", dpi=400)
fig1.savefig(
    fig_dir
    / "comparison"
    / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_zoom.pdf"
)
# fig2.savefig(fig_dir / "comparison" / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_anomalies_zoom.png", dpi=400)
fig2.savefig(
    fig_dir
    / "comparison"
    / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_anomalies_zoom.pdf"
)

#### Multiplicity

In [ ]:
datas = [
    condensation,
    collision_condensation,
    # coalbure_condensation_cke,
    coalbure_condensation_small,
    coalbure_condensation_large,
]

keys = [data.attrs["microphysics_short"] for data in datas]

comparison_dict = multiplicity_comparison_all
compare_key = "condensation"

width_ratios = np.ones(len(keys) + 1)
width_ratios[0] = 0.1
width_ratios = list(width_ratios)

fig1, axs1 = plt.subplots(
    ncols=len(keys) + 1,
    nrows=len(selected_gridboxes),
    figsize=(15, 10),
    sharex=False,
    sharey=False,
    width_ratios=width_ratios,
)
fig2, axs2 = plt.subplots(
    ncols=len(keys) - 1 + 1,
    nrows=len(selected_gridboxes),
    figsize=(12, 10),
    sharex=False,
    sharey=False,
    width_ratios=width_ratios[:-1],
)

for idx_mp, mp in enumerate(keys):
    idx_mp += 1
    for idx_height, sel_gridbox in enumerate(selected_gridboxes):
        data = comparison_dict[mp]
        data = data.sel(gridbox=sel_gridbox).compute()
        diff = data - comparison_dict[compare_key].sel(gridbox=sel_gridbox).compute()
        diff.attrs["units"] = data.attrs["units"]
        diff.attrs["long_name"] = data.attrs["long_name"]

        plot_mean_median_iqr(ax=axs1[idx_height, idx_mp], data=data, dim="cloud_id", x_dim="radius_bins")
        axs1[0, idx_mp].set_title(data_dict[mp]["microphysics"])

        if idx_mp > 1:
            plot_mean_median_iqr(
                ax=axs2[idx_height, idx_mp - 1], data=diff, dim="cloud_id", x_dim="radius_bins"
            )
            axs2[0, idx_mp - 1].set_title(data_dict[mp]["microphysics"])


axs1[0, 1].legend(loc="upper left")
axs2[0, 1].legend(loc="upper left")

for idx, label in enumerate(labels):
    axs1[idx, 0].set_ylabel(label, fontsize=14)
    axs2[idx, 0].set_ylabel(label, fontsize=14)

    axs1[idx, 0].set_xticks([])
    axs2[idx, 0].set_xticks([])
    axs1[idx, 0].set_yticks([])
    axs2[idx, 0].set_yticks([])


# data figure
for ax in axs1[:, 1:].flatten():
    ax.set_xlabel("Radius [µm]")
    ax.set_xscale("log")
    ax.set_yscale("linear")
    ax.set_ylim(0, 80)


# anomaly figure
for ax in axs2[:, 1:].flatten():
    ax.set_xlabel("Radius [µm]")
    ax.set_xscale("log")
    ax.set_yscale("linear")
    ax.set_ylim(-8, 8)

for ax in axs1[:, 1].flatten():
    ax.set_ylabel(label_from_attrs(data))
for ax in axs2[:, 1].flatten():
    ax.set_ylabel(label_from_attrs(diff))

for ax in axs2[:, 1:].flatten():
    ax.axhline(0, color="grey", linestyle="--", alpha=0.5)

fig1.suptitle(f"{label_from_attrs(data)}")
fig2.suptitle(f"Anomalies of {label_from_attrs(diff)} compared to condensation only run")

for fig in [fig1, fig2]:
    fig.tight_layout()

# fig1.savefig(fig_dir / "comparison" / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes.png", dpi=400)
fig1.savefig(
    fig_dir
    / "comparison"
    / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes.pdf"
)
# fig2.savefig(fig_dir / "comparison" / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_anomalies.png", dpi=400)
fig2.savefig(
    fig_dir
    / "comparison"
    / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_anomalies.pdf"
)

for ax in axs1[:, 1:].flatten():
    ax.set_xlim(10, None)
for ax in axs2[:, 1:].flatten():
    ax.set_xlim(10, None)

# fig1.savefig(fig_dir / "comparison" / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_zoom.png", dpi=400)
fig1.savefig(
    fig_dir
    / "comparison"
    / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_zoom.pdf"
)
# fig2.savefig(fig_dir / "comparison" / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_anomalies_zoom.png", dpi=400)
fig2.savefig(
    fig_dir
    / "comparison"
    / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_anomalies_zoom.pdf"
)

### Number concentration

In [ ]:
datas = [
    condensation,
    collision_condensation,
    # coalbure_condensation_cke,
    coalbure_condensation_small,
    coalbure_condensation_large,
]

keys = [data.attrs["microphysics_short"] for data in datas]


comparison_dict = number_superdroplets_comparison_all
compare_key = "condensation"

width_ratios = np.ones(len(keys) + 1)
width_ratios[0] = 0.1
width_ratios = list(width_ratios)

fig1, axs1 = plt.subplots(
    ncols=len(keys) + 1,
    nrows=len(selected_gridboxes),
    figsize=(15, 10),
    sharex=False,
    sharey=False,
    width_ratios=width_ratios,
)
fig2, axs2 = plt.subplots(
    ncols=len(keys) - 1 + 1,
    nrows=len(selected_gridboxes),
    figsize=(12, 10),
    sharex=False,
    sharey=False,
    width_ratios=width_ratios[:-1],
)

for idx_mp, mp in enumerate(keys):
    idx_mp += 1
    for idx_height, sel_gridbox in enumerate(selected_gridboxes):
        data = comparison_dict[mp]
        data = data.sel(gridbox=sel_gridbox).compute()
        diff = data - comparison_dict[compare_key].sel(gridbox=sel_gridbox).compute()
        diff.attrs["units"] = data.attrs["units"]
        diff.attrs["long_name"] = data.attrs["long_name"]

        plot_mean_median_iqr(ax=axs1[idx_height, idx_mp], data=data, dim="cloud_id", x_dim="radius_bins")
        axs1[0, idx_mp].set_title(data_dict[mp]["microphysics"])

        if idx_mp > 1:
            plot_mean_median_iqr(
                ax=axs2[idx_height, idx_mp - 1], data=diff, dim="cloud_id", x_dim="radius_bins"
            )
            axs2[0, idx_mp - 1].set_title(data_dict[mp]["microphysics"])


axs1[0, 1].legend(loc="upper left")
axs2[0, 1].legend(loc="upper left")

for idx, label in enumerate(labels):
    axs1[idx, 0].set_ylabel(label, fontsize=14)
    axs2[idx, 0].set_ylabel(label, fontsize=14)

    axs1[idx, 0].set_xticks([])
    axs2[idx, 0].set_xticks([])
    axs1[idx, 0].set_yticks([])
    axs2[idx, 0].set_yticks([])


# data figure
for ax in axs1[:, 1:].flatten():
    ax.set_xlabel("Radius [µm]")
    ax.set_xscale("log")
    ax.set_yscale("linear")
    ax.set_ylim(-1, 15)


# anomaly figure
for ax in axs2[:, 1:].flatten():
    ax.set_xlabel("Radius [µm]")
    ax.set_xscale("log")
    ax.set_yscale("linear")
    ax.set_ylim(-1, 3)

for ax in axs1[:, 1].flatten():
    ax.set_ylabel(label_from_attrs(data))
for ax in axs2[:, 1].flatten():
    ax.set_ylabel(label_from_attrs(diff))

for ax in axs2[:, 1:].flatten():
    ax.axhline(0, color="grey", linestyle="--", alpha=0.5)

fig1.suptitle(f"{label_from_attrs(data)}")
fig2.suptitle(f"Anomalies of {label_from_attrs(diff)} compared to condensation only run")

for fig in [fig1, fig2]:
    fig.tight_layout()

# fig1.savefig(fig_dir / "comparison" / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes.png", dpi=400)
fig1.savefig(
    fig_dir
    / "comparison"
    / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes.pdf"
)
# fig2.savefig(fig_dir / "comparison" / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_anomalies.png", dpi=400)
fig2.savefig(
    fig_dir
    / "comparison"
    / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_anomalies.pdf"
)

for ax in axs1[:, 1:].flatten():
    ax.set_xlim(10, None)
for ax in axs2[:, 1:].flatten():
    ax.set_xlim(10, None)

# fig1.savefig(fig_dir / "comparison" / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_zoom.png", dpi=400)
fig1.savefig(
    fig_dir
    / "comparison"
    / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_zoom.pdf"
)
# fig2.savefig(fig_dir / "comparison" / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_anomalies_zoom.png", dpi=400)
fig2.savefig(
    fig_dir
    / "comparison"
    / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_anomalies_zoom.pdf"
)

### Mass

In [ ]:
datas = [
    condensation,
    collision_condensation,
    # coalbure_condensation_cke,
    coalbure_condensation_small,
    coalbure_condensation_large,
]

keys = [data.attrs["microphysics_short"] for data in datas]


comparison_dict = mass_comparison_all
compare_key = "condensation"

width_ratios = np.ones(len(keys) + 1)
width_ratios[0] = 0.1
width_ratios = list(width_ratios)

fig1, axs1 = plt.subplots(
    ncols=len(keys) + 1,
    nrows=len(selected_gridboxes),
    figsize=(15, 10),
    sharex=False,
    sharey=False,
    width_ratios=width_ratios,
)
fig2, axs2 = plt.subplots(
    ncols=len(keys) - 1 + 1,
    nrows=len(selected_gridboxes),
    figsize=(12, 10),
    sharex=False,
    sharey=False,
    width_ratios=width_ratios[:-1],
)

for idx_mp, mp in enumerate(keys):
    idx_mp += 1
    for idx_height, sel_gridbox in enumerate(selected_gridboxes):
        data = comparison_dict[mp]
        data = data.sel(gridbox=sel_gridbox).compute()
        diff = data - comparison_dict[compare_key].sel(gridbox=sel_gridbox).compute()
        diff.attrs["units"] = data.attrs["units"]
        diff.attrs["long_name"] = data.attrs["long_name"]

        plot_mean_median_iqr(ax=axs1[idx_height, idx_mp], data=data, dim="cloud_id", x_dim="radius_bins")
        axs1[0, idx_mp].set_title(data_dict[mp]["microphysics"])

        if idx_mp > 1:
            plot_mean_median_iqr(
                ax=axs2[idx_height, idx_mp - 1], data=diff, dim="cloud_id", x_dim="radius_bins"
            )
            axs2[0, idx_mp - 1].set_title(data_dict[mp]["microphysics"])


axs1[0, 1].legend(loc="upper left")
axs2[0, 1].legend(loc="upper left")

for idx, label in enumerate(labels):
    axs1[idx, 0].set_ylabel(label, fontsize=14)
    axs2[idx, 0].set_ylabel(label, fontsize=14)

    axs1[idx, 0].set_xticks([])
    axs2[idx, 0].set_xticks([])
    axs1[idx, 0].set_yticks([])
    axs2[idx, 0].set_yticks([])


# data figure
for ax in axs1[:, 1:].flatten():
    ax.set_xlabel("Radius [µm]")
    ax.set_xscale("log")
    ax.set_yscale("linear")
    ax.set_ylim(0, 5e1)


# anomaly figure
for ax in axs2[:, 1:].flatten():
    ax.set_xlabel("Radius [µm]")
    ax.set_xscale("log")
    ax.set_yscale("linear")
    ax.set_ylim(-2.5, 2.5)
    ax.set_yticks([-2, -1, 0, 1, 2])

for ax in axs1[:, 1].flatten():
    ax.set_ylabel(label_from_attrs(data))
for ax in axs2[:, 1].flatten():
    ax.set_ylabel(label_from_attrs(diff))

for ax in axs2[:, 1:].flatten():
    ax.axhline(0, color="grey", linestyle="--", alpha=0.5)

fig1.suptitle(f"{label_from_attrs(data)}")
fig2.suptitle(f"Anomalies of {label_from_attrs(diff)} compared to condensation only run")

for fig in [fig1, fig2]:
    fig.tight_layout()

# fig1.savefig(fig_dir / "comparison" / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes.png", dpi=400)
fig1.savefig(
    fig_dir
    / "comparison"
    / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes.pdf"
)
# fig2.savefig(fig_dir / "comparison" / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_anomalies.png", dpi=400)
fig2.savefig(
    fig_dir
    / "comparison"
    / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_anomalies.pdf"
)

for ax in axs1[:, 1:].flatten():
    ax.set_xlim(10, None)
for ax in axs2[:, 1:].flatten():
    ax.set_xlim(10, None)

# fig1.savefig(fig_dir / "comparison" / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_zoom.png", dpi=400)
fig1.savefig(
    fig_dir
    / "comparison"
    / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_zoom.pdf"
)
# fig2.savefig(fig_dir / "comparison" / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_anomalies_zoom.png", dpi=400)
fig2.savefig(
    fig_dir
    / "comparison"
    / f"{label_from_attrs(data, return_units=False)}_comparison_selected_gridboxes_anomalies_zoom.pdf"
)

### correlation histograms


In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(10, 5))
for key in mass_comparison_all:
    print(key)
    color = colors_dict[key]
    print(color)
    bins = np.arange(-1, 1, 0.05)

    ax.hist(
        xr.corr(
            latent_heat_comparison_all[key],
            mass_comparison_all[key],
            dim=["radius_bins"],
        ).values.flatten(),
        bins=bins,
        color=color,
        alpha=0.5,
        facecolor=color,
    )

condensation
#CC6677
collision_condensation
#6E9CB3
coalbure_condensation_large
#44AA99
coalbure_condensation_small
#AA4499


In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(10, 5))
for key in mass_comparison_all:
    print(key)
    color = colors_dict[key]
    print(color)
    bins = np.arange(-1, 1, 0.05)

    ax.hist(
        xr.corr(
            latent_heat_comparison_all[key],
            number_superdroplets_comparison_all[key],
            dim=["radius_bins"],
        ).values.flatten(),
        bins=bins,
        color=color,
        alpha=0.5,
        facecolor=color,
    )

condensation
#CC6677
collision_condensation
#6E9CB3
coalbure_condensation_large
#44AA99
coalbure_condensation_small
#AA4499


In [ ]:
corr_dict = dict()

for mp in data_dict:
    print(mp)
    data = data_dict[mp]["dataset"].sel(time=time_slice).sel(cloud_id=cloud_id_selection)
    da_x = 1e3 * data["latent_heating_radius_bins"].mean("time", keep_attrs=True)
    da_x = da_x.where(data["sub_cloud_layer_mask"])
    da_x.attrs["units"] = "$mW m^{-3}$"
    da_x.attrs["long_name"] = "Latent heating"

    da_y = 1e3 * data["mass_represented_per_volume"].mean("time", keep_attrs=True)
    da_y = da_y.where(data["sub_cloud_layer_mask"])
    da_y.attrs["units"] = "$mg m^{-3}$"
    da_y.attrs["long_name"] = "Mass"

    corr_dict[mp] = xr.corr(
        da_x,
        da_y,
        dim=["radius_bins"],
    ).compute()

null_microphysics
condensation
collision_condensation
coalbure_condensation_cke
coalbure_condensation_large
coalbure_condensation_small


In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(10, 5))

for key in corr_dict:
    print(key)
    color = colors_dict[key]
    m_color = adjust_lightness_array([color], 0.9)[0]
    color = adjust_lightness_array([color], 1.3)[0]
    print(color)
    bins = np.arange(-1, 1, 0.05)

    sc = ax.plot(
        corr_dict[key].T,
        data["gridbox_coord3"].T,
        color=color,
        alpha=0.1,
        zorder=0,
    )

    m, s = mean_and_stderror_of_mean(data=corr_dict[key], dims=("cloud_id",))

    ax.plot(
        m,
        data["gridbox_coord3"].mean("cloud_id"),
        color=m_color,
        label=key,
        linewidth=2,
        zorder=2,
    )

    ax.fill_betweenx(
        data["gridbox_coord3"].mean("cloud_id"),
        m - s,
        m + s,
        color=m_color,
        alpha=0.2,
        zorder=1,
    )
ax.set_xlabel("Correlation")
ax.set_ylabel("Height [m]")

null_microphysics
#a6a6a6
condensation
#e3abb4
collision_condensation
#a7c3d1
coalbure_condensation_cke
#dfa85a
coalbure_condensation_large
#70c6b7
coalbure_condensation_small
#c670b7


Text(0, 0.5, 'Height [m]')

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(10, 5))
for key in mass_comparison_all:
    print(key)
    color = colors_dict[key]
    print(color)
    bins = np.arange(-1, 1, 0.05)
    res = xr.corr(
        latent_heat_comparison_all[key],
        mass_comparison_all[key],
        dim=["radius_bins"],
    )

    ax.plot(res.T, res["gridbox"].T, color=color, alpha=0.01)
    ax.plot(res.mean("cloud_id"), res["gridbox"], color=color, linestyle="-", alpha=1);

condensation
#CC6677
collision_condensation
#6E9CB3
coalbure_condensation_large
#44AA99
coalbure_condensation_small
#AA4499


In [ ]:
keys = mass_comparison_all.keys()

fig, axs = plt.subplots(ncols=len(keys), figsize=(12, 3))
for i, key in enumerate(keys):
    print(key)
    color = colors_dict[key]
    print(color)
    bins = np.arange(-1, 1, 0.05)

    axs[i].hist(
        xr.corr(
            latent_heat_comparison_all[key],
            mass_comparison_all[key],
            dim=["radius_bins"],
        ).values.flatten(),
        bins=bins,
        color=color,
        alpha=0.5,
        facecolor=color,
    )

condensation
#CC6677


collision_condensation
#6E9CB3
coalbure_condensation_large
#44AA99
coalbure_condensation_small
#AA4499


In [ ]:
keys = mass_comparison_all.keys()

fig, axs = plt.subplots(ncols=len(keys), figsize=(12, 3))
for i, key in enumerate(keys):
    print(key)
    color = colors_dict[key]
    print(color)
    bins = np.arange(-1, 1, 0.05)

    axs[i].hist(
        xr.corr(
            latent_heat_comparison_all[key],
            multiplicity_comparison_all[key],
            dim=["radius_bins"],
        ).values.flatten(),
        bins=bins,
        color=color,
        alpha=0.5,
        facecolor=color,
    )

condensation
#CC6677
collision_condensation
#6E9CB3
coalbure_condensation_large
#44AA99
coalbure_condensation_small
#AA4499


# Temporal evolution of the distributions

In [ ]:
def temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax):
    over_color = (211 / 255, 246 / 255, 50 / 255, 0.9)
    under_color = (84 / 255, 218 / 255, 255 / 255, 0.1)
    my_cmap = mpl.colormaps["plasma"].resampled(15)
    my_cmap.set_over(over_color)
    my_cmap.set_under(under_color)
    my_norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

    ds = data_dict[key]["dataset"].sel(cloud_id=int(cloud_id))
    ds = ds.sel(gridbox=slice(0, ds["max_gridbox"]))
    fig, axs = plt.subplots(
        ncols=2, nrows=1, figsize=(15, 7), sharex=False, sharey=True, width_ratios=[1, 0.5]
    )

    clabel = label_from_attrs(ds[variable])

    pcm = axs[0].pcolormesh(
        ds[variable]["time"],
        ds["gridbox_coord3"],
        ds[variable].sum("radius_bins").compute().T,
        shading="auto",
        cmap=my_cmap,
        norm=my_norm,
    )

    fig.colorbar(mappable=pcm, ax=axs[0], label=clabel, extend="both")
    axs[0].set_xlabel("Time $s$")
    axs[0].set_ylabel("Height $m$")

    ds_stationary = ds.sel(time=slice(400, None))

    axs[1].plot(
        ds_stationary[variable].sum("radius_bins").quantile(0.5, "time").compute(),
        ds_stationary["gridbox_coord3"],
    )
    axs[1].plot(
        ds_stationary[variable].sum("radius_bins").T,
        ds_stationary["gridbox_coord3"],
        color=[0.7, 0.7, 0.7],
        linewidth=0.1,
        alpha=0.1,
        zorder=0,
    )
    axs[1].fill_betweenx(
        ds_stationary["gridbox_coord3"],
        ds_stationary[variable].sum("radius_bins").quantile(0.25, "time").compute(),
        ds_stationary[variable].sum("radius_bins").quantile(0.75, "time").compute(),
        color="k",
        alpha=0.1,
    )
    axs[1].set_xlim(vmin, vmax)
    axs[1].set_xlabel(clabel)

    axs[1].set_ylabel("Height $m$")

    fig.suptitle(
        f"Cloud {cloud_id} - {data_dict[key]['microphysics']} - {label_from_attrs(ds[variable], return_units = False)}"
    )

    return fig, axs


def temporal_evolution_different_gridboxes(
    data_dict, key, variable, cloud_id, vmin, vmax, isel_gridboxes=[-1, -2, -3, -5, 10, 0]
):
    over_color = (211 / 255, 246 / 255, 50 / 255, 0.9)
    under_color = (84 / 255, 218 / 255, 255 / 255, 0.1)
    my_cmap = mpl.colormaps["plasma"].resampled(15)
    my_cmap.set_over(over_color)
    my_cmap.set_under(under_color)
    my_norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

    ds = data_dict[key]["dataset"].sel(cloud_id=int(cloud_id))
    ds = ds.sel(gridbox=slice(0, ds["max_gridbox"]))

    clabel = label_from_attrs(ds[variable])

    gridboxes = ds["gridbox"].isel(gridbox=isel_gridboxes).compute()

    width_ratios = np.ones(len(gridboxes) + 1)
    width_ratios[-1] = 0.1

    fig, axs = plt.subplots(
        ncols=len(gridboxes) + 1,
        nrows=1,
        figsize=(15, 10),
        sharex=False,
        sharey=False,
        width_ratios=width_ratios,
    )

    for idx, gridbox in enumerate(gridboxes):
        altitude = ds["gridbox_coord3"].sel(gridbox=gridbox).compute()
        ds_selected = ds.sel(gridbox=gridbox)
        pcm = axs[idx].pcolormesh(
            ds_selected[variable]["radius_bins"],
            ds_selected[variable]["time"],
            ds_selected[variable].compute(),
            shading="auto",
            cmap=my_cmap,
            norm=my_norm,
        )
        axs[idx].set_title(f"Gridbox {gridbox.data}\n{altitude.data}m")
    for ax in axs[:-1]:
        ax.set_xscale("log")
        ax.set_xticks([1e0, 1e1, 1e2, 1e3])
        ax.set_xlim(1e-1, 1e3)
        ax.grid()
        ax.set_xlabel("Radius $\\mu m$")

    for ax in axs[1:-1]:
        ax.set_yticklabels([])

    axs[0].set_ylabel("Time $s$")

    fig.colorbar(
        cax=axs[-1],
        mappable=pcm,
        orientation="vertical",
        label=clabel,
        extend="both",
    )
    fig.suptitle(
        f"Cloud {cloud_id} - {data_dict[key]['microphysics']} - {label_from_attrs(ds[variable], return_units = False)}"
    )

    return fig, axs

#### Multiplicity

In [ ]:
vmin, vmax = 1e-10, 1000
vmin_small, vmax_small = vmin, 50
cloud_id = 142
variable = "xi_per_volume"

keys = [
    "null_microphysics",
    "condensation",
    "collision_condensation",
    # "coalbure_condensation_cke",
    "coalbure_condensation_large",
    "coalbure_condensation_small",
]
isel_gridboxes = [-1, -2, 15, 1]
for key in keys:
    # fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
    # fig.savefig(fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}.png", dpi=600)
    # fig.savefig(fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}.pdf")
    fig, axs = temporal_evolution_different_gridboxes(
        data_dict=data_dict,
        key=key,
        variable=variable,
        cloud_id=cloud_id,
        vmin=vmin_small,
        vmax=vmax_small,
        isel_gridboxes=isel_gridboxes,
    )
    fig.savefig(
        fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}_different_gridboxes.png",
        dpi=200,
    )
    # fig.savefig(fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}_different_gridboxes.pdf")

### Mass

In [ ]:
vmin, vmax = 1e-10, 0.2
vmin_small, vmax_small = vmin, 0.015
variable = "mass_represented_per_volume"
cloud_id = 142

keys = [
    "null_microphysics",
    "condensation",
    "collision_condensation",
    # "coalbure_condensation_cke",
    "coalbure_condensation_large",
    "coalbure_condensation_small",
]

isel_gridboxes = [-1, -2, 15, 1]
for key in keys:
    # fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
    # fig.savefig(fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}.png", dpi=600)
    # fig.savefig(fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}.pdf")
    fig, axs = temporal_evolution_different_gridboxes(
        data_dict=data_dict,
        key=key,
        variable=variable,
        cloud_id=cloud_id,
        vmin=vmin_small,
        vmax=vmax_small,
        isel_gridboxes=isel_gridboxes,
    )
    fig.savefig(
        fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}_different_gridboxes.png",
        dpi=200,
    )
    # fig.savefig(fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}_different_gridboxes.pdf")

### Number of SDs

In [ ]:
vmin, vmax = 1e-10, 40
vmin_small, vmax_small = vmin, 2
variable = "number_superdroplets_per_volume"
cloud_id = 142

keys = [
    "null_microphysics",
    "condensation",
    "collision_condensation",
    # "coalbure_condensation_cke",
    "coalbure_condensation_large",
    "coalbure_condensation_small",
]

isel_gridboxes = [-1, -2, 15, 1]
for key in keys:
    # fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
    # fig.savefig(fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}.png", dpi=600)
    # fig.savefig(fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}.pdf")
    fig, axs = temporal_evolution_different_gridboxes(
        data_dict=data_dict,
        key=key,
        variable=variable,
        cloud_id=cloud_id,
        vmin=vmin_small,
        vmax=vmax_small,
        isel_gridboxes=isel_gridboxes,
    )
    fig.savefig(
        fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}_different_gridboxes.png",
        dpi=200,
    )
    # fig.savefig(fig_dir / "temporal_evolution" / f"{variable}_{key}_cloud_{cloud_id}_different_gridboxes.pdf")

### Latent heating

In [ ]:
vmin, vmax = -0.02, -1e-10
variable = "latent_heating_radius_bins"
cloud_id = 222
key = "condensation"

# fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
fig, axs = temporal_evolution_different_gridboxes(data_dict, key, variable, cloud_id, vmin, vmax)

In [ ]:
vmin, vmax = -0.02, -1e-10
variable = "latent_heating_radius_bins"
cloud_id = 222
key = "collision_condensation"

# fig, axs = temporal_evolution_of_variable(data_dict, key, variable, cloud_id, vmin, vmax)
fig, axs = temporal_evolution_different_gridboxes(data_dict, key, variable, cloud_id, vmin, vmax)